In [1]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 9.6 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
train = load_dataset("stsb_multi_mt", name="es", split="dev")

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Dataset stsb_multi_mt downloaded and prepared to /root/.cache/huggingface/datasets/stsb_multi_mt/es/1.0.0/a5d260e4b7aa82d1ab7379523a005a366d9b124c76a5a5cf0c4c5365458b0ba9. Subsequent calls will reuse this data.


In [3]:
print(type(train))
print(type(train[0]))
print(type(train['sentence1']))
print(type(train['similarity_score']))
print(train)
print(train[0])
print(train[1238])
print(train[0]["sentence1"])

<class 'datasets.arrow_dataset.Dataset'>
<class 'dict'>
<class 'list'>
<class 'list'>
Dataset({
    features: ['sentence1', 'sentence2', 'similarity_score'],
    num_rows: 1500
})
{'sentence1': 'Un hombre con un casco está bailando.', 'sentence2': 'Un hombre con un casco está bailando.', 'similarity_score': 5.0}
{'sentence1': 'Los equipos trabajaron para instalar una nueva alcantarilla y preparar la autopista para que los automovilistas pudieran usar los carriles hacia el este para viajar, ya que las nubes de tormenta amenazaban con arrojar más lluvia.', 'sentence2': 'Los equipos trabajaron para instalar una nueva alcantarilla y repavimentar la autopista para que los automovilistas pudieran usar los carriles hacia el este para viajar.', 'similarity_score': 3.3329999446868896}
Un hombre con un casco está bailando.


In [4]:
import re

In [5]:
sentence1 = []
sentence2 = []
sim_scores = []

for length in range (len(train)):
  sentence1.append(train[length]["sentence1"].lower())
  sentence2.append(train[length]["sentence2"].lower())
  sim_scores.append(train[length]["similarity_score"])

for sentence in sentence1:
      # Puntuation 
    sentence = re.sub(r"[^\w\s']+" ,' ', sentence)
    # sentence = re.sub(r"(?<=[^\d\s])\.(?=\s|$)", "", sentence)
    # sentence = re.sub(r"(?<=\w)\.(?=\w)", "", sentence)
    # sentence = re.sub(re".","", sentence)

    #Removing extra spaces
    sentence = re.sub(r"\s+", " ", sentence)

In [6]:
from spacy.lang.es import Spanish
nlp = Spanish()

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [7]:
tokenized_sentences1 = []
for sentence in sentence1:
    doc = nlp(sentence)
    tokenized_sentences1.append([token.text for token in doc])

# print(tokenized_sentences1)

[['un', 'hombre', 'con', 'un', 'casco', 'está', 'bailando', '.'], ['un', 'niño', 'pequeño', 'está', 'montando', 'a', 'caballo', '.'], ['un', 'hombre', 'está', 'alimentando', 'a', 'una', 'serpiente', 'con', 'un', 'ratón', '.'], ['una', 'mujer', 'está', 'tocando', 'la', 'guitarra', '.'], ['una', 'mujer', 'está', 'tocando', 'la', 'flauta', '.'], ['una', 'mujer', 'está', 'cortando', 'una', 'cebolla', '.'], ['un', 'hombre', 'está', 'borrando', 'una', 'pizarra', '.'], ['una', 'mujer', 'lleva', 'un', 'niño', '.'], ['tres', 'hombres', 'están', 'tocando', 'la', 'guitarra', '.'], ['una', 'mujer', 'pela', 'una', 'patata', '.'], ['la', 'gente', 'está', 'jugando', 'al', 'cricket', '.'], ['un', 'hombre', 'está', 'tocando', 'una', 'guitarra', '.'], ['el', 'puma', 'está', 'persiguiendo', 'al', 'oso', '.'], ['el', 'hombre', 'cortó', 'un', 'árbol', 'con', 'un', 'hacha', '.'], ['el', 'hombre', 'está', 'tocando', 'la', 'guitarra', '.'], ['un', 'hombre', 'está', 'encontrando', 'algo', '.'], ['la', 'chica',

In [8]:
tokenized_sentences2 = []
for sentence in sentence2:
    doc = nlp(sentence)
    tokenized_sentences2.append([token.text for token in doc])

# print(tokenized_sentences2)

[['un', 'hombre', 'con', 'un', 'casco', 'está', 'bailando', '.'], ['un', 'niño', 'está', 'montando', 'a', 'caballo', '.'], ['el', 'hombre', 'está', 'alimentando', 'a', 'la', 'serpiente', 'con', 'un', 'ratón', '.'], ['un', 'hombre', 'está', 'tocando', 'la', 'guitarra', '.'], ['un', 'hombre', 'está', 'tocando', 'una', 'flauta', '.'], ['un', 'hombre', 'está', 'cortando', 'cebollas', '.'], ['el', 'hombre', 'está', 'borrando', 'la', 'pizarra', '.'], ['una', 'mujer', 'está', 'llevando', 'a', 'su', 'bebé', '.'], ['tres', 'hombres', 'están', 'en', 'el', 'escenario', 'tocando', 'la', 'guitarra', '.'], ['una', 'mujer', 'está', 'pelando', 'una', 'patata', '.'], ['los', 'hombres', 'están', 'jugando', 'al', 'cricket', '.'], ['un', 'hombre', 'está', 'tocando', 'una', 'flauta', '.'], ['un', 'puma', 'está', 'persiguiendo', 'a', 'un', 'oso', '.'], ['un', 'hombre', 'corta', 'un', 'árbol', 'con', 'un', 'hacha', '.'], ['un', 'hombre', 'está', 'tocando', 'una', 'guitarra', '.'], ['una', 'mujer', 'está', 'c

In [9]:
import gensim
import numpy as np
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
%cd /content/drive/My Drive/NLP/PreTrained_models

/content/drive/My Drive/NLP/PreTrained_models


In [12]:
# !wget http://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.bin.gz

In [13]:
# !gunzip SBW-vectors-300-min5.bin.gz

In [14]:
model = KeyedVectors.load_word2vec_format('SBW-vectors-300-min5.bin', binary=True)
# SBW-vectors-300-min5

In [15]:
sentence_embeddings1 = []

for sentence in tokenized_sentences1:
    sentence_embedding = []
    for word in sentence:
        if word in model:
            sentence_embedding.append(model[word])
    if sentence_embedding:
        sentence_embedding = np.mean(sentence_embedding, axis=0)
    else:
        sentence_embedding = np.zeros(model.vector_size)
    sentence_embeddings1.append(sentence_embedding)
# print(sentence_embeddings[0])
# print((sentence_embeddings[0].shape))

In [16]:
sentence_embeddings2 = []

for sentence in tokenized_sentences2:
    sentence_embedding = []
    for word in sentence:
        if word in model:
            sentence_embedding.append(model[word])
    if sentence_embedding:
        sentence_embedding = np.mean(sentence_embedding, axis=0)
    else:
        sentence_embedding = np.zeros(model.vector_size)
    sentence_embeddings2.append(sentence_embedding)
# print(sentence_embeddings[0])
# print((sentence_embeddings[0].shape))

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
# sentence_embeddings1 and sentence_embeddings2 are the two lists of sentence embeddings
# of the same length
assert len(sentence_embeddings1) == len(sentence_embeddings2)

# Compute the cosine similarity between corresponding indexed sentences
cos_sim = []
for emb1, emb2 in zip(sentence_embeddings1, sentence_embeddings2):
    similarity = cosine_similarity(emb1.reshape(1, -1), emb2.reshape(1, -1))[0][0]
    cos_sim.append(similarity)


In [19]:
# print(cos_sim[1])


0.9863051


In [22]:
from scipy.stats import pearsonr

# cos_sim and similarity_score are the two lists of similarity scores
# of the same length
assert len(cos_sim) == len(sim_scores)

# Calculate the Pearson correlation coefficient and its p-value
corr, p_value = pearsonr(cos_sim, sim_scores)

print(f"Pearson correlation coefficient: {corr:.3f}")
print(f"P-value: {p_value:.3f}")


Pearson correlation coefficient: 0.633
P-value: 0.000
